# Mini-BMN Notebook

Expand mini-BMN hamiltonian into Pauli strings

To Do
- add comments
- check units
- how to simulate? what to simulate?
- fuzzy spheres

In [1]:
import qiskit
import numpy as np
import sympy as sp
from collections import Counter
from qiskit.circuit import Parameter, ParameterVector
from qiskit.quantum_info import Pauli, Operator, SparsePauliOp
from symengine.lib.symengine_wrapper import Zero as spZero
from utils import SpecialUnitaryGroup, annihilation_operator_old, creation_operator_old

In [2]:
import matplotlib
import matplotlib.pyplot as plt
from cycler import cycler

plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.major.size'] = 5.0
plt.rcParams['xtick.minor.size'] = 3.0
plt.rcParams['ytick.major.size'] = 5.0
plt.rcParams['ytick.minor.size'] = 3.0
plt.rc('font', family='serif',size=14)
#matplotlib.rc('text', usetex=True)
matplotlib.rc('legend', fontsize=14)
plt.rcParams['ytick.minor.size'] = 3.0
matplotlib.rcParams.update({"axes.grid" : True,
                            "grid.alpha": 0.75,
                            "grid.linewidth": 0.5})
matplotlib.rcParams['axes.prop_cycle'] = cycler(color=['#E24A33', '#348ABD', '#988ED5', '#777777', '#FBC15E', '#8EBA42', '#FFB5B8'])
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [3]:
number_of_matrices = 3
gauge_group_degree = 2 # the N in SU(N)
number_of_generators = gauge_group_degree**2 - 1 #understand the diff b/w U(N) and SU(N)

bits_per_oscillator = 2
number_states_per_oscillator = 2**bits_per_oscillator

number_qubits = (
    number_of_matrices
    * number_of_generators
    * bits_per_oscillator
)

print(f'number_qubits {number_qubits}, Hilbert space dimension {2**number_qubits}')

number_qubits 18, Hilbert space dimension 262144


In [4]:
su_group = SpecialUnitaryGroup(gauge_group_degree)
su_group.structure_constants

{(1, 2, 3): 1.0,
 (1, 3, 2): -1.0,
 (2, 1, 3): -1.0,
 (2, 3, 1): 1.0,
 (3, 1, 2): 1.0,
 (3, 2, 1): -1.0}

In [5]:
index_map = {}
counter = 0
for i in range(number_of_matrices):
    for a in range(number_of_generators):
        index_map[(i,a)] = counter
        counter += bits_per_oscillator
index_map_inverse = dict(map(reversed, index_map.items()))
index_map

{(0, 0): 0,
 (0, 1): 2,
 (0, 2): 4,
 (1, 0): 6,
 (1, 1): 8,
 (1, 2): 10,
 (2, 0): 12,
 (2, 1): 14,
 (2, 2): 16}

In [38]:
# Eq. 4.5 of https://arxiv.org/pdf/2011.06573
single_qubit_state_map = {
    '00' : SparsePauliOp(data=["I", "Z"], coeffs=np.array([0.5, 0.5])),
    '01' : SparsePauliOp(data=["X", "Y"], coeffs=np.array([0.5, 0.5 * 1j])),
    '10' : SparsePauliOp(data=["X", "Y"], coeffs=np.array([0.5, -0.5 * 1j])),
    '11' : SparsePauliOp(data=["I", "Z"], coeffs=np.array([0.5, -0.5])),
    }
single_qubit_state_map

{'00': SparsePauliOp(['I', 'Z'],
               coeffs=[0.5+0.j, 0.5+0.j]),
 '01': SparsePauliOp(['X', 'Y'],
               coeffs=[0.5+0.j , 0. +0.5j]),
 '10': SparsePauliOp(['X', 'Y'],
               coeffs=[0.5+0.j , 0. -0.5j]),
 '11': SparsePauliOp(['I', 'Z'],
               coeffs=[ 0.5+0.j, -0.5+0.j])}

In [52]:
def annihilation_operator(matrix_idx: int, generator_idx: int) -> SparsePauliOp:
    """
    Build the annihilation operator for the oscillator.

    Parameters
    ----------
    matrix_idx : int
        The SO(3) index enumerating the matrices (1, 2, 3)
    generator_idx : int
        The generator index (1, 2, ..., N^2)

    Returns
    -------
    SparsePauliOp
        The annihilation operator
    """
    qubit_indx0 = index_map[(matrix_idx, generator_idx)]
    operator = 0
    for j in range(number_states_per_oscillator-1):
        bra_bitstring = f'{j:0{bits_per_oscillator}b}'
        ket_bitstring = f'{j+1:0{bits_per_oscillator}b}'
        for i in range(bits_per_oscillator):
            new_term = single_qubit_state_map[bra_bitstring[i] + ket_bitstring[i]]
            new_term = new_term.subs('I', f'I{qubit_indx0+i}')
            new_term = new_term.subs('X', f'X{qubit_indx0+i}')
            new_term = new_term.subs('Y', f'Y{qubit_indx0+i}')
            new_term = new_term.subs('Z', f'Z{qubit_indx0+i}')
            operator += new_term
    return operator.simplify()


def creation_operator(matrix_idx: int, generator_idx: int) -> SparsePauliOp:
    """
    Build the annihilation operators for the oscillator.

    Parameters
    ----------
    matrix_idx : int
        The SO(3) index enumerating the matrices (1, 2, 3)
    generator_idx : int
        The generator index (1, 2, ..., N^2)

    Returns
    -------
    SparsePauliOp
        The creation operator.
    """
    qubit_index_0 = index_map[(matrix_idx, generator_idx)] # first non-trivial qubit index
    operator = SparsePauliOp(data="I" * number_qubits, coeffs=np.asarray([0])) # initialize operator

    # loop over the truncated set of internal states for each oscillator
    for j in range(number_states_per_oscillator-1):

        # Eq. 4.3 of https://arxiv.org/pdf/2011.06573
        bra_bitstring = f'{j+1:0{bits_per_oscillator}b}'
        ket_bitstring = f'{j:0{bits_per_oscillator}b}'

        # loop over bits per oscillaotr
        for i in range(bits_per_oscillator):
            qubit_index = qubit_index_0 + i
            single_qubit_matrix_element = single_qubit_state_map[bra_bitstring[i] + ket_bitstring[i]]
            first_pauli = single_qubit_matrix_element.paulis[0].__str__()
            second_pauli = single_qubit_matrix_element.paulis[1].__str__()
            first_pauli = ("I" * qubit_index) + first_pauli + (number_qubits - qubit_index - 1) * "I"
            second_pauli = ("I" * qubit_index) + second_pauli + (number_qubits - qubit_index - 1) * "I"
            operator += SparsePauliOp(data=[first_pauli, second_pauli], coeffs=single_qubit_matrix_element.coeffs)
    return operator.simplify()


def annihilation_operator(matrix_idx: int, generator_idx: int) -> SparsePauliOp:
    """
    Build the annihilation operators for the oscillator.

    Parameters
    ----------
    matrix_idx : int
        The SO(3) index enumerating the matrices (1, 2, 3)
    generator_idx : int
        The generator index (1, 2, ..., N^2)

    Returns
    -------
    SparsePauliOp
        The creation operator.
    """
    return creation_operator(matrix_idx: int, generator_idx: int)

In [53]:
creation_operator_old(0, 0)

1.0*I0 + 0.5*X0 + 1.5*X1 - 0.5*I*Y0 - 0.5*I*Y1

In [54]:
print(creation_operator(0, 0))

SparsePauliOp(['I', 'Z'],
              coeffs=[0.5+0.j, 0.5+0.j])
SparsePauliOp(['X', 'Y'],
              coeffs=[0.5+0.j , 0. -0.5j])
SparsePauliOp(['X', 'Y'],
              coeffs=[0.5+0.j , 0. -0.5j])
SparsePauliOp(['X', 'Y'],
              coeffs=[0.5+0.j , 0. +0.5j])
SparsePauliOp(['I', 'Z'],
              coeffs=[ 0.5+0.j, -0.5+0.j])
SparsePauliOp(['X', 'Y'],
              coeffs=[0.5+0.j , 0. -0.5j])
SparsePauliOp(['IIIIIIIIIIIIIIIIII', 'IXIIIIIIIIIIIIIIII', 'IYIIIIIIIIIIIIIIII', 'XIIIIIIIIIIIIIIIII', 'YIIIIIIIIIIIIIIIII'],
              coeffs=[1. +0.j , 1.5+0.j , 0. -0.5j, 0.5+0.j , 0. -0.5j])


In [ ]:
SparsePauliOp(data=["I", "Z"], coeffs=np.array([0.5, 0.5])).coeffs

In [ ]:
creation_operator(0, 0)

In [ ]:
creation_operator(0, 0)

In [ ]:
string_to_Pauli_map = {s:Pauli(s) for s in "IXYZ"}

In [ ]:
def sympy_Pauli_to_qiskit_Pauli(operator_string):
    '''CAREFUL WITH ENDIAN CONVENTION'''
    pauli_string = operator_string[0]
    qubit_number = int(operator_string[1:])

    #print(pauli_string, qubit_number)

    if qubit_number == 0:
        new_operator = string_to_Pauli_map[pauli_string]
    else:
        new_operator = Pauli("I")

    for i in range(1, number_qubits):
        if i != qubit_number:
            new_operator = new_operator ^ Pauli("I")
        else:
            new_operator = new_operator ^ string_to_Pauli_map[pauli_string]
    return new_operator#.reduce()

def sympy_Pauli_to_qiskit_Pauli(operator_string):
    return SparsePauliOp(data=operator_string)

In [ ]:
sympy_Pauli_to_qiskit_Pauli("IXI")

In [ ]:
def sympy_operator_to_qiskit_operator(sympy_operator):
    operator = 0 #sympy_Pauli_to_qiskit_Pauli(f'I{number_qubits}')
    for arg in sympy_operator.args:
        #print(arg)
        if len(arg.args) == 2:
            coeff = complex(arg.args[0])
        elif len(arg.args) == 3:
            coeff = complex(sp.prod(arg.args[0:2]))
        else:
            raise ValueError
        #print(coeff, sympy_Pauli_to_qiskit_Pauli(str(arg.args[-1])))
        operator += coeff * sympy_Pauli_to_qiskit_Pauli(str(arg.args[-1]))
    return operator

In [ ]:
sympy_operator_to_qiskit_operator("IXI")

In [ ]:
def position_operator(i, A):
    creation = sympy_operator_to_qiskit_operator(creation_operator(i, A))
    annihilation = sympy_operator_to_qiskit_operator(annihilation_operator(i, A))
    coeff = complex(np.sqrt(1/2))
    operator = coeff * (creation + annihilation)
    return operator#.reduce()


def momentum_operator(i, A):
    creation = sympy_operator_to_qiskit_operator(creation_operator(i, A))
    annihilation = sympy_operator_to_qiskit_operator(annihilation_operator(i, A))
    coeff = complex(-1j * np.sqrt(1/2))
    operator = coeff * (creation - annihilation)
    return operator#.reduce()


def hamiltonian_bosonic_free(sqrt_nu):
    operator = 0 #0*sympy_Pauli_to_qiskit_Pauli(f'I{number_qubits}')
    for i in range(number_of_matrices):
        for a in range(number_of_generators):
            operator += 0.5 * (
                momentum_operator(i, a) @ momentum_operator(i, a)
                + position_operator(i, a) @ position_operator(i, a)
                )
    #return operator.reduce() * (sqrt_nu * sqrt_nu)
    return operator * (sqrt_nu * sqrt_nu)

def hamiltonian_cubic_interaction(sqrt_nu):
    operator = 0
    for a in range(number_of_generators):
        for b in range(number_of_generators):
            for c in range(number_of_generators):
                coeff = float(-6 * su_group.structure_constants.get((a+1,b+1,c+1), 0))
                if coeff != 0:
                    operator += coeff*(position_operator(0, a)
                             @ position_operator(1, b)
                             @ position_operator(2, c)
                             )#.reduce()
    #return operator.reduce() / sqrt_nu
    return operator / sqrt_nu

def hamiltonian_quartic_interaction(sqrt_nu):
    operator = 0

    for i in range(number_of_matrices):
        for j in range(number_of_matrices):

            for a in range(number_of_generators):
                for b in range(number_of_generators):
                    for c in range(number_of_generators):
                        for d in range(number_of_generators):

                                coeff = float((1/4) * sum(
                                    su_group.structure_constants.get((a+1,b+1,e+1), 0)
                                    * su_group.structure_constants.get((c+1,d+1,e+1), 0)
                                    for e in range(number_of_generators)
                                ))

                                if coeff != 0:
                                    operator += coeff * (
                                        position_operator(i, a)
                                        @ position_operator(j, b)
                                        @ position_operator(i, c)
                                        @ position_operator(j, d)
                                        )#.reduce()

    return operator.reduce() * (1 / (sqrt_nu*sqrt_nu))

In [ ]:
def hamiltonian(sqrt_nu, free_only=False):
    return (
        hamiltonian_bosonic_free(sqrt_nu)
        + int(not free_only) * hamiltonian_cubic_interaction(sqrt_nu)
        + int(not free_only) * hamiltonian_quartic_interaction(sqrt_nu)
        )

In [ ]:
sqrt_nu = Parameter('sqrt_nu')
H = hamiltonian(sqrt_nu, free_only=True)

terms = {0:'free', 1:'cubic', 2:'quartic'}
for key, value in terms.items():
    print(f'number of terms in H_{value}: {len(H[key])}')

In [ ]:
H_dict = {}
H_weights_dict = {}

for i in range(len(H)):

    for x in H[i].primitive:
        assert len(x.paulis) == 1
        H_dict[x.paulis[0]] = H_dict.get(x.paulis[0], 0) + (complex(x.coeffs[0]) * H[i].coeff)

print(f'total number of terms in H: {len(H_dict)}')

In [ ]:
number_qubits

In [ ]:
hamiltonian(sqrt_nu);

In [ ]:
def weight(pauli):
    weight = 0
    for key, value in Counter(str(pauli)).items():
        if key != 'I':
            weight += value
    return weight

In [ ]:
Counter([weight(pauli) for pauli in H_dict.keys()])

In [ ]:
mat = 0 * Pauli(number_qubits * 'I').to_matrix(sparse=True)
for key, value in H_dict.items():
    mat += key.to_matrix(sparse=True) #* value.sympify().subs('sqrt_nu', 1)

In [ ]:
from scipy.sparse.linalg import eigs
eigs(mat, k=3, which='SM')

In [ ]:
193331200/(262144 * 262144)

In [ ]:
hamiltonian(sqrt_nu, free_only=True).reduce()

In [ ]:
eigvals, eigvecs = np.linalg.eig(mat)

In [ ]:
from matplotlib.ticker import FormatStrFormatter

fig, ax = plt.subplots()
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.hist(np.real(eigvals)/sqrt_nu**2)
plt.show()